In [1]:
def ER_import(stoch_base_ER):
    read_in = open('import_dates.csv')
    myreader = csv.reader(read_in)
    
    for row in myreader:
        mat_gen_dates = row[:]
    
    ERs = np.genfromtxt('ER_list_' + str(stoch_base_ER) + '_' + str(mat_gen_dates[-1]) + '.csv',
                                  delimiter = ',')
        
    return ERs

In [2]:
def CN_stoch_in_fxn(ERs_all):
    CN_import = np.empty((total_time, 4, scenarios))
    for i in range(scenarios):
        CN_import[:,:3,i] = np.genfromtxt('STO_C14_Be10_ER' + str(ERs_all[i]) + '_expmat.csv', 
                                          delimiter = ',')

        CN_import[:,3,i] = np.genfromtxt('STO_He3_ER' + str(ERs_all[i]) + '_surfmat.csv',
                              delimiter = ',')
        
    return CN_import

In [2]:
def sampling_fxn(CN_import_matrix, sample_no, scenarios):
    sampling_years = np.sort(np.random.choice(np.arange(int(total_time * 0.1),total_time),sample_no))
    sampling_mat = np.empty((sample_no, 6, scenarios))
    measured = np.empty((6,scenarios))
    
    for j in range(scenarios):
        
        sampling_14C = []
        sampling_Be = []
        sampling_rat = []
        sampling_3He = []
    
        for i in range(sample_no):           

            sampling_14C.append(CN_import_matrix[sampling_years[i],0,j])
            sampling_Be.append(CN_import_matrix[sampling_years[i],1,j])
            sampling_rat.append(CN_import_matrix[sampling_years[i],2,j])
            sampling_3He.append(CN_import_matrix[sampling_years[i],3,j])
        
        sampling_mat[:,0,j] = sampling_14C
        sampling_mat[:,1,j] = sampling_Be
        sampling_mat[:,2,j] = sampling_rat
        sampling_mat[:,3,j] = sampling_3He
        sampling_mat[:,4,j] = np.divide(sampling_3He,sampling_Be) * (SurfBe10 / P0_He3)
        sampling_mat[:,5,j] = sampling_years
        
        
        measured[0,j] = np.mean(sampling_mat[:,0,j])
        measured[1,j] = np.mean(sampling_mat[:,1,j])
        measured[2,j] = measured[0,j] / measured[1,j]
        measured[3,j] = np.mean(sampling_mat[:,3,j])
        measured[4,j] = (np.mean(sampling_mat[:,4,j]) / np.mean(sampling_mat[:,1])) * (SurfBe10 / P0_He3)
        measured[5,j] = (CN_lambda / rock_rho * (SurfBe10 / measured[1,j] - BeDecayfct))
    
    measured_ER = [(CN_lambda / rock_rho * (SurfBe10 / measured[1,j] - BeDecayfct)) for j in range(scenarios)]
    
    return sampling_mat, measured, measured_ER  

In [3]:
def generate_constant_comps(measured_ER):
    %run CRN_comparison_outputs_generator.ipynb
    %run He3_comparison_outputs_generator.ipynb
    
    comp_mat = np.empty((total_time, 5, scenarios))
    
    shift_ER = False
    save_only_surf = True
    save_output = True    
    
    for i in range(scenarios):
        
        initial_ER = measured_ER[i]
        comp_mat[:,2,i], comp_mat[:,0,i], comp_mat[:,1,i] = CRN_comp_loop_fxn(total_time, initial_ER)[2:5] 
        comp_mat[:,3,i] = He3_comp_loop_fxn(total_time, initial_ER)
        comp_mat[:,4,i] = np.divide(comp_mat[:,3,i], comp_mat[:,1,i]) * (P0_Be10 / P0_He3)
        
    return comp_mat

In [5]:
def original_ER_fxn(ERs_all):
    %run CRN_comparison_outputs_generator.ipynb
    %run He3_comparison_outputs_generator.ipynb

    og_ER_mat = np.empty((total_time, 5, scenarios))
    
    shift_ER = False
    save_only_surf = True
    save_output = True    
        
    for i in range(scenarios):
        initial_ER = ERs_all[i]
        og_ER_mat[:,2,i], og_ER_mat[:,0,i], og_ER_mat[:,1,i] = CRN_comp_loop_fxn(total_time, initial_ER)[2:5] 
        og_ER_mat[:,3,i] = He3_comp_loop_fxn(total_time, initial_ER)
        og_ER_mat[:,4,i] = np.divide(og_ER_mat[:,3,i], og_ER_mat[:,1,i]) * (SurfBe10 / P0_He3)
        
    return og_ER_mat

In [14]:
def compare_fxn():
    
    ERs_all = ER_import(stoch_base_ER)
    CN_import = CN_stoch_in_fxn(ERs_all)
    samples, measured, measured_ER = sampling_fxn(CN_import, sample_no, scenarios)
    const_comp_mat = generate_constant_comps(measured_ER)
    
    return const_comp_mat, samples, measured, measured_ER